In [145]:
import pandas as pd
import os
from sklearn.linear_model import LogisticRegressionCV
from sklearn.feature_selection import f_classif
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
import pickle
import numpy as np

In [97]:
df = pd.read_csv('../data/combined_dataset.csv')

Encoding team names

In [98]:
team_encoded = pd.get_dummies(df['team'], prefix='team')
team_decoded = pd.get_dummies(team_encoded).idxmax(1)

In [99]:
opp_team_encoded = pd.get_dummies(df['oppteam'], prefix='oppteam')
opp_team_decoded = pd.get_dummies(opp_team_encoded).idxmax(1)

Encoding Location

In [100]:
loc_encoded = pd.get_dummies(df['loc'], prefix='loc')
loc_decoded = pd.get_dummies(loc_encoded).idxmax(1)

In [101]:
df_encoded = pd.concat([df.drop(['team', 'loc', 'oppteam'], axis=1), team_encoded, loc_encoded, opp_team_encoded], axis=1)

In [127]:
df_most_recent = df_encoded.loc[df_encoded['Season'] > 2011].reset_index(drop=True)
df_most_recent = df_most_recent.dropna(axis=1, how='all')
df_most_recent = df_most_recent.drop(['score', 'oppscore'], axis=1)

Partition dataset into train and test

In [128]:
bool_2016 = df_most_recent['Season'] == 2016
x_train, y_train, x_test, y_test = (df_most_recent.loc[~bool_2016].drop(['Season', 'game_won'], axis=1),
                                    df_most_recent.loc[~bool_2016, 'game_won'],
                                    df_most_recent.loc[bool_2016].drop(['Season', 'game_won'], axis=1),
                                    df_most_recent.loc[bool_2016, 'game_won'])

In [129]:
train_univariate = f_classif(x_train, y_train)

C:\Users\burksa\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:113: UserWarning: Features [ 40  58 199 352 398 416 557 710] are constant.
  UserWarning)


In [130]:
pd.DataFrame({'Columns': x_train.columns,
              'pval': train_univariate[1]}).sort_values(by='pval')

,Columns,pval
371,loc_A,0.000000e+00
1,fgm,0.000000e+00
5,ftm,0.000000e+00
6,fta,0.000000e+00
372,loc_H,0.000000e+00
8,dr,0.000000e+00
9,ast,0.000000e+00
12,blk,0.000000e+00
13,pf,0.000000e+00
3,fgm3,4.676559e-311


In [142]:
clf = LogisticRegressionCV(Cs=np.logspace(-8, 1, 10), cv=3, verbose=True, n_jobs=-1)
clf.fit(x_train, y_train)

[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  1.2min finished


LogisticRegressionCV(Cs=array([  1.00000e-08,   1.00000e-07,   1.00000e-06,   1.00000e-05,
         1.00000e-04,   1.00000e-03,   1.00000e-02,   1.00000e-01,
         1.00000e+00,   1.00000e+01]),
           class_weight=None, cv=3, dual=False, fit_intercept=True,
           intercept_scaling=1.0, max_iter=100, multi_class='ovr',
           n_jobs=-1, penalty='l2', random_state=None, refit=True,
           scoring=None, solver='lbfgs', tol=0.0001, verbose=True)

In [146]:
with open('../model/logcv_model.p', 'wb') as f:
    pickle.dump(clf, f)

In [143]:
clf.score(x_train, y_train)

0.89859442408498402

In [144]:
clf.score(x_test, y_test)

0.88245033112582782